# Reaching multiple targets with a manipulator
The objective of this exercise is to reach multiple targets with a manipulator.

We provide a basic example for reaching one point, and you have to modify it for sequence of multiple targets. Below it is the basic example, there we'll guide you to the final result.


In [5]:
# %load arm_example.py
from crocoddyl import *
import pinocchio
import numpy as np

robot = loadTalosArm()
robot.initDisplay(loadModel=True)

robot.q0.flat[:] = [  2,1.5,-2,0,0,0,0 ]
robot.model.armature[:] = .2
frameId = robot.model.getFrameId('gripper_left_joint')
DT = 1e-2
T  = 25

target = np.array([ 0.4,0.,0.4 ])

robot.viewer.gui.addSphere('world/point',.1,[1,0,0,1])  # radius = .1, RGBA=1001
robot.viewer.gui.applyConfiguration('world/point', target.tolist()+[0,0,0,1] )  # xyz+quaternion
robot.viewer.gui.refresh()

# Create the cost model
costTrack = CostModelFrameTranslation(robot.model,frame=frameId,ref=target)
costXReg  = CostModelState(robot.model,StatePinocchio(robot.model))
costUReg  = CostModelControl(robot.model)

# Create the action model with empty costs
model     = DifferentialActionModel(robot.model)
termmodel = DifferentialActionModel(robot.model)

# Then let's add the running and terminal cost functions
model.costs.addCost( name="pos", weight = 1, cost = costTrack)
model.costs.addCost( name="xreg", weight = 1e-4, cost = costXReg)
model.costs.addCost( name="ureg", weight = 1e-7, cost = costUReg)

termmodel.costs.addCost( name="pos", weight = 1000, cost = costTrack)
termmodel.costs.addCost( name="xreg", weight = 1e-4, cost = costXReg)
termmodel.costs.addCost( name="ureg", weight = 1e-7, cost = costUReg)

# Create the problem
x0 = np.concatenate([ m2a(robot.q0), np.zeros(robot.model.nv)])
problem = ShootingProblem(x0, [ IntegratedActionModelEuler(model) ] * T,
                          IntegratedActionModelEuler(termmodel))

# Creating the DDP solver for this OC problem, defining a logger
ddp = SolverDDP(problem)
cameraTF = [2., 2.68, 0.54, 0.2, 0.62, 0.72, 0.22]
ddp.callback = [ CallbackDDPVerbose(), CallbackSolverDisplay(robot,4,cameraTF) ]

# Solving it with the DDP algorithm
ddp.solve()

# Visualizing the solution in gepetto-viewer
for x in ddp.xs: robot.display(a2m(x))

print('Finally reached = ',
      ddp.datas()[T].differential.costs['pos'].pinocchio.oMf[frameId].translation.T)


iter 	 cost 	      theta 	    gamma 	  muV 	      muLM 	 alpha
   0  4.03759e+02  4.63479e-05  3.91802e+02  1.00000e-09  1.00000e-09  1.0000
   1  2.63286e+02  7.37335e-05  6.42758e+02  1.00000e-09  1.00000e-09  0.2500
   2  2.20577e+02  4.50380e-05  3.86323e+02  1.00000e-09  1.00000e-09  1.0000
   3  1.18659e+02  4.71622e-05  3.86564e+02  1.00000e-09  1.00000e-09  1.0000
   4  1.05175e+02  1.38233e-05  1.20665e+02  1.00000e-09  1.00000e-09  1.0000
   5  7.09226e+01  2.10900e-05  1.79137e+02  1.00000e-09  1.00000e-09  1.0000
   6  3.00234e+01  1.56809e-05  1.35583e+02  1.00000e-09  1.00000e-09  1.0000
   7  1.12396e+01  5.11524e-06  4.39839e+01  1.00000e-09  1.00000e-09  1.0000
   8  1.05845e+01  2.77501e-07  2.34366e+00  1.00000e-09  1.00000e-09  1.0000
   9  1.03916e+01  1.19610e-07  1.02553e+00  1.00000e-09  1.00000e-09  0.2500
iter 	 cost 	      theta 	    gamma 	  muV 	      muLM 	 alpha
  10  1.03757e+01  3.38563e-08  2.90998e-01  1.00000e-09  1.00000e-09  1.0000
  11  1.03267e+0

## I. DifferentialActionModel for Pinocchio ABA
This scenario uses an action model that computes 2nd order differential dynamics with Pinocchio. Note that it can accept several cost models. This action model is tailored for robot applications, and at the same time, it's modular since:
 - you can modify the robot dynamics by changing Pinocchio model, and
 - you can formulate any cost function by simply adding running a terminal costs.

## II. Cost models

A cost model computes a scalar cost value and its gradient and Hessian. All the models implemented are computing a cost residual and are computing the Hessian with the Gauss approximation.

We implemented reusable cost models for controlling 
 - a frame placement (translation or velocity),
 - the center of mass position, and 
 - state  and control spaces.

In the example above, we used the CostModelFrameTranslation which defines a 3d position task, and the state and control regularizers.

As for any cost model in crocoddyl, if you write your own cost model you need to create a data class for your cost function. The cost data must be created from a pinocchio data (the rational is that the pinocchio data used to compute the dynamics should be re-used to compute the cost).


In [6]:
dataTrack = costTrack.createData(robot.data)

### II.a Frame position cost

You define a frame ID and the reference position as a 3D array. The cost is the distance between the frame and the target. This cost depends on $\mathbf{x}$ (specifically the configuration $\mathbf{q}$). You can double check the 0s in its gradient.

In [7]:
pinocchio.updateFramePlacements(robot.model,robot.data)
pinocchio.computeJointJacobians(robot.model,robot.data,a2m(x))
costTrack.calcDiff(dataTrack,x0,None)
print(dataTrack.Lx,dataTrack.Lu)

(array([-8.72516471e-03,  5.88825598e-03, -2.05031219e-04, -9.92433215e-05,
        1.43041681e-03, -7.60960409e-05,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00]), 0)


### II.b State cost
In this part of the tutorial you must define a State model. It defines 
 - the dimension of the state and its tangent, and
 - the exponential/integrate and difference/log operators.
The operators can described using Pinocchio functions. And the exercite consists on adding them into your State class. Please note crocoddyl has abstract functions for this.

The state cost uses a reference in state space (State.zero() by default). The cost is the distance, computed with state.difference between the current state and the reference. Hence, with this cost, we regularize both position and velocity.

### II.c Control cost

The control cost uses a control reference as in the state cost. The cost is the distance the current control and the reference. Hence the cost regularizes torque commands.

### II.d Add cost models to the differential action model
Each time we want to include a new cost function, we use addCost function inside our DAM. In this function you're also able its weight.

## III. Create the problem with integrated action model
Differential action models describe cost and dynamics in continuous-time, however our optimal control solvers work in discrete-time. We have created the integrated action model in order to deal with this.

In the previous code, we have used an abstract class that uses simpletic Euler rules. In the cartpole exercise you have learnt how to use integrated action models for your problem.

## IV. Callbacks

Callback functions are needed for analysing and debugging the performance of the solver for your specific problem.
For problems defined with Pinocchio, you can display the robot trajectory per each iterate by including CallbackSolverDisplay. With this callback, you can display robot motions with different rates. Additionally, CallbackDDPVerbose prints a message that allows us to understand the behaviour of the solver.

Generally speaking, an user is able to describe any callback function. This function will be run once per iterate and it has access to all data.

## VI. Modifying the example

Start by defining several targets (let's say 4 targets, all at x=0.4, and at y and z being either 0 or 0.4), and display then in the viewer.


The shooting problem will be composed of 4 sequences of action models. Each sequence consists on T shooting "running" nodes and 1 terminal node. The running nodes mostly have regularization terms, while the terminal nodes have a strong cost toward the respective target.

[ R1,R1,R1 ... R1,T1, R2,R2 .... R2, T2, R3 ... R3, T3, R4 ... R4 ] , T4

First create 4 running models and 4 terminal models.

Then you need to add a position cost, and state and control regularization to each running action model. Please  note that for terminal action model is only needed the position cost. Additionally, in the running models, the position cost should be low, and it should be high in the terminal models.

Now create a shooting problem.

In [ ]:
seq0 = [runningmodels[0]]*T + [terminalmodel[0]]
seq1 = [runningmodels[1]]*T + [terminalmodel[1]]
seq2 = [runningmodels[2]]*T + [terminalmodel[2]]
seq3 = [runningmodels[3]]*T 
problem = ShootingProblem(x0,seq0+seq1+seq2+seq3,terminalmodel[3])

Create a DDP solver for this problem and run it. 

In [ ]:
ddp = SolverDDP(problem)
ddp.solve()

Well, it should not work, at least no on the first shot. The DDP solver is likely not strong enough to accept the random weights that you have selected. 

If it is working nicely from the first shot, display it in the viewer and go take a coffee. But you will likely have to tweak the gains to make it work.

**It is suggested to first optimize only sequence 1. When you are happy with it, add sequence 2 and optimize again, etc.**


## V. Penalty
The solver works with double precisions, so it is quite robust to high weight. 10000 is likely to be accepted for example. But if you make the problem too difficult, the solver will break. 

In that case, you can implement a simple penalty solver by setting the weight to be 10**i, and creating a for loop to explore i from 0 to 5. At each iteration of the loop, run the solver from the previous solution and for few iterations only.

In [8]:
for i in range(1,6):
    terminalmodel[1].costs['pos'].weight = 10**i
    ddp.solve(init_xs=ddp.xs,init_us=ddp.us,maxiter=10)


NameError: name 'terminalmodel' is not defined